In [5]:
#Run this in your machine before working with the model to install it:
!pip install pygsl

In [7]:
import nest
nest.set_verbosity('M_ERROR')

from pynestml.frontend.pynestml_frontend import generate_nest_target


generate_nest_target(input_path="models/adex_gamma_E.nestml",   # file containing the neuron model definition
                     suffix="_ml",                               # append to model name to avoid clash with existing model
                     module_name="m1module",                     # enumerate modules; name must end in "module"
                     target_path="tgt",                          # path for auxiliary files generated by NESTML
                     install_path=".")                           # location for generated module; must be "." on EBRAINS

nest.Install('m1module')


              -- N E S T --
  Copyright (C) 2004 The NEST Initiative

 Version: 3.4
 Built: Jun 28 2023 05:25:40

 This program is provided AS IS and comes with
 NO WARRANTY. See the file LICENSE for details.

 Problems or suggestions?
   Visit https://www.nest-simulator.org

 Type 'nest.help()' to find out more about NEST.

ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
ANTLR runtime and generated code versions disagree: 4.13.0!=4.10.1
AN

GeneratedCodeBuildException: Error occurred during cmake! More detailed error messages can be found in stderr.

In [2]:
import nest.raster_plot
import nest.voltage_trace
import numpy as np
import matplotlib.pyplot as plt
nest.ResetKernel()
nest.resolution = 0.1
%matplotlib inline
rt = nest.GetDefaults('adex_gamma_E_ml', 'receptor_types')

duration = 1000

# Define pop numbers
N_E = 160
N_I = 40

net = (2*N_E)+(2*N_I)

# Define Synaptic Weights
nmdaW = 1.0
ampaW = 5
gabaW = -3.34

# Set non-uniform initial membrane voltage
VmProbE = np.random.uniform(low=-65, high=-50, size=N_E)
VmProbI= np.random.uniform(low=-65, high=-50, size=N_I)

# Define syn params
stim_syn = {'weight': nest.random.lognormal(mean=0.5, std=0.1), 'receptor_type': rt['AMPA'], 'delay':1.5}
noise_synE = {'weight': nest.random.lognormal(mean=0.2, std=0.05), 'receptor_type': rt['AMPA'], 'delay':1.5}
noise_synI = {'weight': nest.random.lognormal(mean=-0.2, std=0.05), 'receptor_type': rt['GABA'], 'delay':1.5}
EEsyn = nest.CollocatedSynapses({'weight': ampaW, 'receptor_type': rt['AMPA'], 'delay': 1.5},
                                {'weight': 0.4, 'receptor_type': rt['NMDA'], 'delay': 1.5})
EIsyn = nest.CollocatedSynapses({'weight': ampaW, 'receptor_type': rt['AMPA'], 'delay': 1.5},
                                {'weight': 0.55, 'receptor_type': rt['NMDA'], 'delay': 1.5})

localIsyn = {'weight': gabaW, 'receptor_type': rt['GABA'], 'delay': 1.5}
distIsyn = {'weight': gabaW, 'receptor_type': rt['GABA'], 'delay': 1.5}


# Define conn params
connEE = {'rule': 'pairwise_bernoulli', 'p': 0.65, 'allow_autapses': False}
connEI = {'rule': 'pairwise_bernoulli', 'p': 0.5}
connIE = {'rule': 'pairwise_bernoulli', 'p': 0.6}
connStim = {'rule': 'pairwise_bernoulli', 'p': 0.2}


# Create cells
E1 = nest.Create('adex_gamma_E_ml', N_E, params={'V_m': VmProbE})
E2 = nest.Create('adex_gamma_E_ml', N_E, params={'V_m': VmProbE})
I1 = nest.Create('adex_gamma_E_ml', N_I, params={'V_th':-47.5, 'V_peak':-37.5,'a': 0, 'b':0,'Delta_T': 0.5, 'V_m': VmProbI})
I2 = nest.Create('adex_gamma_E_ml', N_I, params={'V_th':-47.5, 'V_peak':-37.5,'a': 0, 'b':0,'Delta_T': 0.5, 'V_m': VmProbI})

# Create devices
pg = nest.Create("poisson_generator", {'rate':30000})
noise1 = nest.Create("poisson_generator", {'rate':1500})
noise2 = nest.Create("poisson_generator", {'rate':1500})
E1spk = nest.Create('spike_recorder')
I1spk = nest.Create('spike_recorder')
E2spk = nest.Create('spike_recorder')
I2spk = nest.Create('spike_recorder')
vM = nest.Create('voltmeter')

# Connect devices
nest.Connect(E1, E1spk)
nest.Connect(I1, I1spk)
nest.Connect(E2, E2spk)
nest.Connect(I2, I2spk)
nest.Connect(vM, E1)

# stim syns
nest.Connect(pg, E1, syn_spec=stim_syn, conn_spec=connStim)
nest.Connect(pg, I1, syn_spec=stim_syn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.125})
# nest.Connect(noise1, E1, syn_spec=noise_synE)
# nest.Connect(noise1, I1, syn_spec=noise_synE)
# nest.Connect(noise1, E2, syn_spec=noise_synE)
# nest.Connect(noise1, I2, syn_spec=noise_synE)
# nest.Connect(noise1, E1, syn_spec=noise_synI)
# nest.Connect(noise1, I1, syn_spec=noise_synI)
# nest.Connect(noise1, E2, syn_spec=noise_synI)
# nest.Connect(noise1, I2, syn_spec=noise_synI)

# EE syns
nest.Connect(E1, E2, syn_spec=EEsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.8,  'allow_autapses': False})
nest.Connect(E1, E1, syn_spec=EEsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.215,  'allow_autapses': False})
nest.Connect(E2, E1, syn_spec=EEsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.45,  'allow_autapses': False}) 
nest.Connect(E2, E2, syn_spec=EEsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.3,  'allow_autapses': False})

# EI syns
nest.Connect(E1, I2, syn_spec=EIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.41,  'allow_autapses': False}) 
nest.Connect(E1, I1, syn_spec=EIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.395,  'allow_autapses': False}) 
nest.Connect(E2, I1, syn_spec=EIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.425,  'allow_autapses': False}) 
nest.Connect(E2, I2, syn_spec=EIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.365,  'allow_autapses': False}) 

# IE syns
nest.Connect(I1, E1, syn_spec=localIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.38,  'allow_autapses': False}) #prev 0.385
nest.Connect(I1, E2, syn_spec=distIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.235,  'allow_autapses': False}) 
nest.Connect(I2, E1, syn_spec=distIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.4,  'allow_autapses': False}) #prev 0.425
nest.Connect(I2, E2, syn_spec=localIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.315,  'allow_autapses': False}) 

# II syns
# nest.Connect(I1, I2, syn_spec=distIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.15})
# nest.Connect(I2, I1, syn_spec=distIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.15})
nest.Connect(I2, I2, syn_spec=localIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.275,  'allow_autapses': False})
nest.Connect(I1, I1, syn_spec=localIsyn, conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.275,  'allow_autapses': False})

NESTErrors.UnknownComponent: UnknownComponent in SLI function GetDefaults_l: /adex_gamma_E_ml is not a known component.

In [ ]:
nest.Simulate(duration)

In [ ]:
spkct = len(E2spk.events['times'])+len(E1spk.events['times'])+len(I2spk.events['times'])+len(I1spk.events['times'])
rate_net = spkct / duration * 1000.0 / net
rate_net

In [ ]:
timesE1 = nest.GetStatus(E1spk, "events")[0]["times"]
sendersE1 = nest.GetStatus(E1spk, "events")[0]["senders"]

timesE2 = nest.GetStatus(E2spk, "events")[0]["times"]
sendersE2 = nest.GetStatus(E2spk, "events")[0]["senders"]

timesI1 = nest.GetStatus(I1spk, "events")[0]["times"]
sendersI1 = nest.GetStatus(I1spk, "events")[0]["senders"]

timesI2 = nest.GetStatus(I2spk, "events")[0]["times"]
sendersI2 = nest.GetStatus(I2spk, "events")[0]["senders"]

# Create a raster plot with different colors for each population
plt.figure(figsize=(10, 5))
plt.scatter(timesE1, sendersE1, marker="|", color="blue", label="E1")
plt.scatter(timesE2, sendersE2, marker="|", color="green", label="E2")  
plt.scatter(timesI1, sendersI1, marker="|", color="red", label="I1")
plt.scatter(timesI2, sendersI2, marker="|", color="orange", label="I2") 
plt.title("Raster Plot of Neuron Populations")
plt.xlabel("Time (ms)")
plt.ylabel("Neuron ID")
# plt.xlim(0, 1000)
# plt.ylim(0, 200)  # Adjust the y-axis limit based on the total number of neurons
plt.legend(loc='lower right')
plt.show()

In [ ]:
NeuronIDE1 = np.array(sendersE1)
NeuronIDI1 = np.array(sendersI1)

timeE1=np.array(timesE1) #time in ms
timeI1=np.array(timesI1)

X, Y = np.random.rand(2, 400) * np.array([[max, min]]).T

time_resolution = 1  # time resolution
npts = int(duration / time_resolution)  # nb points in LFP vector

xe = max / 2
ye = max / 2  # coordinates of electrode

va = 200  # axonal velocity (mm/sec)
lambda_ = 0.2  # space constant (mm)
dur = 100  # total duration of LFP waveform
nlfp = int(dur / time_resolution)  # nb of LFP pts
amp_e = 0.7  # uLFP amplitude for exc cells
amp_i = -3.4  # uLFP amplitude for inh cells
sig_i = 2.1  # std-dev of ihibition (in ms)
sig_e = 1.5 * sig_i  # std-dev for excitation

# amp_e = -0.16	# exc uLFP amplitude (deep layer)
# amp_i = -0.2	# inh uLFP amplitude (deep layer)

amp_e = 0.48  # exc uLFP amplitude (soma layer)
amp_i = 3  # inh uLFP amplitude (soma layer)

# amp_e = 0.24	# exc uLFP amplitude (superficial layer)
# amp_i = -1.2	# inh uLFP amplitude (superficial layer)

# amp_e = -0.08	# exc uLFP amplitude (surface)
# amp_i = 0.3	# inh uLFP amplitude (surface)

dist = np.sqrt((X - xe) ** 2 + (Y - ye) ** 2)  # distance to  electrode in mm
delay = 10.4 + dist / va  # delay to peak (in ms)
amp = np.exp(-dist / lambda_)
amp[:N_E] *= amp_e
amp[N_I:] *= amp_i

s_e = 2 * sig_e * sig_e
s_i = 2 * sig_i * sig_i


Time_LFP= (np.arange(npts) * time_resolution) #in ms

#=======================================

def f_temporal_kernel(t, tau):
    """function defining temporal part of the kernel"""
    return np.exp(-(t ** 2) / tau)


#=======================================
def calc_lfp(cells_time,cells_id,tau):
    """Calculate LFP from cells"""

    # this is a vectorised computation and as such it might be memory hungry
    # for long LFP series/large number of cells it may be more efficient to calculate it through looping

    spt = cells_time
    cid = cells_id
    
    kernel_contribs = amp[None, cid] * f_temporal_kernel(
        Time_LFP[:, None] - delay[None, cid] - spt[None, :], tau
    )
    lfp = kernel_contribs.sum(1)
    return lfp

#=======================================

lfp_E = calc_lfp(timeE1,NeuronIDE1,s_e)
lfp_I = calc_lfp(timeI1,NeuronIDI1,s_i)

LFP = lfp_E + lfp_I 
